In [1]:
cd ../src

/home/user/Desktop/Kaggle/Script/NLP/FPrize Effective Arguments/src


In [3]:
import torch
import random
import numpy as np
import pandas as pd
from pathlib import Path
# from utils.viz import visualize

from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModel, AutoConfig

%env TOKENIZERS_PARALLELISM = true


import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
warnings.simplefilter("ignore")

env: TOKENIZERS_PARALLELISM=true


test/ - A folder containing an example essay from the test set. The actual test set comprises about 3,000 essays in a format similar to the training set essays.
The test set essays are distinct from the training set essays.

# Params

In [4]:
ROOT_DATA = Path('../data/')
data_folder = ROOT_DATA/"feedback-prize-effectiveness"
kfold_name = "fold_k_5_seed_42"
TARGET = ['Ineffective','Adequate','Effective']

# Data

In [5]:
train_df = pd.read_csv(data_folder/"psl_train.csv")
train_df.shape

(144287, 110)

In [6]:
train_df["discourse_effectiveness"]='Ineffective'

In [7]:
train_df.head(2)

,essay_id,discourse_id,discourse_idx,dbl_ac_fold_0_Ineffective,dbl_ac_fold_0_Adequate,dbl_ac_fold_0_Effective,dbl_ac_fold_1_Ineffective,dbl_ac_fold_1_Adequate,dbl_ac_fold_1_Effective,dbl_ac_fold_2_Ineffective,dbl_ac_fold_2_Adequate,dbl_ac_fold_2_Effective,dbl_ac_fold_3_Ineffective,dbl_ac_fold_3_Adequate,dbl_ac_fold_3_Effective,dbl_ac_fold_4_Ineffective,dbl_ac_fold_4_Adequate,dbl_ac_fold_4_Effective,discourse_idx.1,dbl_ac_awp_fold_0_Ineffective,dbl_ac_awp_fold_0_Adequate,dbl_ac_awp_fold_0_Effective,dbl_ac_awp_fold_1_Ineffective,dbl_ac_awp_fold_1_Adequate,dbl_ac_awp_fold_1_Effective,dbl_ac_awp_fold_2_Ineffective,dbl_ac_awp_fold_2_Adequate,dbl_ac_awp_fold_2_Effective,dbl_ac_awp_fold_3_Ineffective,dbl_ac_awp_fold_3_Adequate,dbl_ac_awp_fold_3_Effective,dbl_ac_awp_fold_4_Ineffective,dbl_ac_awp_fold_4_Adequate,dbl_ac_awp_fold_4_Effective,discourse_idx.2,dbxl_ac_fold_0_Ineffective,dbxl_ac_fold_0_Adequate,dbxl_ac_fold_0_Effective,dbxl_ac_fold_1_Ineffective,dbxl_ac_fold_1_Adequate,dbxl_ac_fold_1_Effective,dbxl_ac_fold_2_Ineffective,dbxl_ac_fold_2_Adequate,dbxl_ac_fold_2_Effective,dbxl_ac_fold_3_Ineffective,dbxl_ac_fold_3_Adequate,dbxl_ac_fold_3_Effective,dbxl_ac_fold_4_Ineffective,dbxl_ac_fold_4_Adequate,dbxl_ac_fold_4_Effective,discourse_idx.3,dbl_cd_fold_0_Ineffective,dbl_cd_fold_0_Adequate,dbl_cd_fold_0_Effective,dbl_cd_fold_1_Ineffective,dbl_cd_fold_1_Adequate,dbl_cd_fold_1_Effective,dbl_cd_fold_2_Ineffective,dbl_cd_fold_2_Adequate,dbl_cd_fold_2_Effective,dbl_cd_fold_3_Ineffective,dbl_cd_fold_3_Adequate,dbl_cd_fold_3_Effective,dbl_cd_fold_4_Ineffective,dbl_cd_fold_4_Adequate,dbl_cd_fold_4_Effective,discourse_idx.4,dbxl_cd_fold_0_Ineffective,dbxl_cd_fold_0_Adequate,dbxl_cd_fold_0_Effective,dbxl_cd_fold_1_Ineffective,dbxl_cd_fold_1_Adequate,dbxl_cd_fold_1_Effective,dbxl_cd_fold_2_Ineffective,dbxl_cd_fold_2_Adequate,dbxl_cd_fold_2_Effective,dbxl_cd_fold_3_Ineffective,dbxl_cd_fold_3_Adequate,dbxl_cd_fold_3_Effective,dbxl_cd_fold_4_Ineffective,dbxl_cd_fold_4_Adequate,dbxl_cd_fold_4_Effective,discourse_idx.5,hf43_dr_fold_0_Ineffective,hf43_dr_fold_0_Adequate,hf43_dr_fold_0_Effective,hf43_dr_fold_1_Ineffective,hf43_dr_fold_1_Adequate,hf43_dr_fold_1_Effective,hf43_dr_fold_2_Ineffective,hf43_dr_fold_2_Adequate,hf43_dr_fold_2_Effective,hf43_dr_fold_3_Ineffective,hf43_dr_fold_3_Adequate,hf43_dr_fold_3_Effective,hf43_dr_fold_4_Ineffective,hf43_dr_fold_4_Adequate,hf43_dr_fold_4_Effective,fold_k_5_seed_42,fold_k_5_seed_2020,fold_k_8_seed_42,fold_k_8_seed_2020,fold_k_10_seed_42,fold_k_10_seed_2020,target,Ineffective,Adequate,Effective,discourse_text,discourse_type,discourse_effectiveness
0,423A1CA112E2,1622627660524,1078c641ef02,0.034966,0.954007,0.011027,0.018977,0.963046,0.017976,0.015324,0.967579,0.017097,0.042056,0.941951,0.015993,0.038436,0.953973,0.007592,1078c641ef02,0.035761,0.953399,0.01084,0.067797,0.895976,0.036227,0.085879,0.883173,0.030948,0.078403,0.907612,0.013985,0.062033,0.927665,0.010302,1078c641ef02,0.032995,0.954974,0.012031,0.032231,0.952640,0.015129,0.042135,0.946509,0.011356,0.015863,0.955661,0.028476,0.019859,0.975816,0.004325,1078c641ef02,0.017759,0.961099,0.021142,0.078091,0.907290,0.014619,0.037290,0.911325,0.051385,0.038027,0.955099,0.006874,0.037068,0.954009,0.008923,1078c641ef02,0.024958,0.960981,0.014062,0.055443,0.936815,0.007742,0.048652,0.947200,0.004149,0.026633,0.949759,0.023609,0.023523,0.967084,0.009393,NaN,0.026505,0.945362,0.028132,0.035739,0.945428,0.018833,0.031681,0.939529,0.028790,0.019637,0.962005,0.018357,0.024594,0.963408,0.011998,4.0,2.0,5.0,4.0,8.0,8.0,1,0.038277,0.944679,0.017044,Modern humans today are always on their phone....,Lead,Ineffective
1,423A1CA112E2,1622627653021,3695c85c6495,0.197380,0.798637,0.003982,0.076309,0.918173,0.005518,0.124825,0.872499,0.002675,0.220555,0.776049,0.003396,0.202561,0.795569,0.001870,3695c85c6495,0.108020,0.889190,0.00279,0.114678,0.881405,0.003916,0.138686,0.855806,0.005508,0.145438,0.850422,0.004140,0.175941,0.820701,0.003357,3695c85c6495,0.056605,0.939123,0.00

In [8]:
train_df.loc[train_df[kfold_name]!=10].target.value_counts()

1    20972
2     9326
0     6461
Name: target, dtype: int64

In [9]:
for c in TARGET:
    cols = [x for x in train_df.columns if x!=c  if c in x if 'fold_0' in x]
    print(cols)
    train_df[c] = train_df[cols].mean(axis=1)

['dbl_ac_fold_0_Ineffective', 'dbl_ac_awp_fold_0_Ineffective', 'dbxl_ac_fold_0_Ineffective', 'dbl_cd_fold_0_Ineffective', 'dbxl_cd_fold_0_Ineffective', 'hf43_dr_fold_0_Ineffective']
['dbl_ac_fold_0_Adequate', 'dbl_ac_awp_fold_0_Adequate', 'dbxl_ac_fold_0_Adequate', 'dbl_cd_fold_0_Adequate', 'dbxl_cd_fold_0_Adequate', 'hf43_dr_fold_0_Adequate']
['dbl_ac_fold_0_Effective', 'dbl_ac_awp_fold_0_Effective', 'dbxl_ac_fold_0_Effective', 'dbl_cd_fold_0_Effective', 'dbxl_cd_fold_0_Effective', 'hf43_dr_fold_0_Effective']


In [12]:
df_fold0 = train_df[train_df[kfold_name]==0]
df_fold0[kfold_name] = 10

In [13]:
train_df.loc[train_df[kfold_name]!=10,TARGET] = pd.get_dummies(train_df.loc[train_df[kfold_name]!=10].target).values

In [14]:
train_df.head(2)

,essay_id,discourse_id,discourse_idx,dbl_ac_fold_0_Ineffective,dbl_ac_fold_0_Adequate,dbl_ac_fold_0_Effective,dbl_ac_fold_1_Ineffective,dbl_ac_fold_1_Adequate,dbl_ac_fold_1_Effective,dbl_ac_fold_2_Ineffective,dbl_ac_fold_2_Adequate,dbl_ac_fold_2_Effective,dbl_ac_fold_3_Ineffective,dbl_ac_fold_3_Adequate,dbl_ac_fold_3_Effective,dbl_ac_fold_4_Ineffective,dbl_ac_fold_4_Adequate,dbl_ac_fold_4_Effective,discourse_idx.1,dbl_ac_awp_fold_0_Ineffective,dbl_ac_awp_fold_0_Adequate,dbl_ac_awp_fold_0_Effective,dbl_ac_awp_fold_1_Ineffective,dbl_ac_awp_fold_1_Adequate,dbl_ac_awp_fold_1_Effective,dbl_ac_awp_fold_2_Ineffective,dbl_ac_awp_fold_2_Adequate,dbl_ac_awp_fold_2_Effective,dbl_ac_awp_fold_3_Ineffective,dbl_ac_awp_fold_3_Adequate,dbl_ac_awp_fold_3_Effective,dbl_ac_awp_fold_4_Ineffective,dbl_ac_awp_fold_4_Adequate,dbl_ac_awp_fold_4_Effective,discourse_idx.2,dbxl_ac_fold_0_Ineffective,dbxl_ac_fold_0_Adequate,dbxl_ac_fold_0_Effective,dbxl_ac_fold_1_Ineffective,dbxl_ac_fold_1_Adequate,dbxl_ac_fold_1_Effective,dbxl_ac_fold_2_Ineffective,dbxl_ac_fold_2_Adequate,dbxl_ac_fold_2_Effective,dbxl_ac_fold_3_Ineffective,dbxl_ac_fold_3_Adequate,dbxl_ac_fold_3_Effective,dbxl_ac_fold_4_Ineffective,dbxl_ac_fold_4_Adequate,dbxl_ac_fold_4_Effective,discourse_idx.3,dbl_cd_fold_0_Ineffective,dbl_cd_fold_0_Adequate,dbl_cd_fold_0_Effective,dbl_cd_fold_1_Ineffective,dbl_cd_fold_1_Adequate,dbl_cd_fold_1_Effective,dbl_cd_fold_2_Ineffective,dbl_cd_fold_2_Adequate,dbl_cd_fold_2_Effective,dbl_cd_fold_3_Ineffective,dbl_cd_fold_3_Adequate,dbl_cd_fold_3_Effective,dbl_cd_fold_4_Ineffective,dbl_cd_fold_4_Adequate,dbl_cd_fold_4_Effective,discourse_idx.4,dbxl_cd_fold_0_Ineffective,dbxl_cd_fold_0_Adequate,dbxl_cd_fold_0_Effective,dbxl_cd_fold_1_Ineffective,dbxl_cd_fold_1_Adequate,dbxl_cd_fold_1_Effective,dbxl_cd_fold_2_Ineffective,dbxl_cd_fold_2_Adequate,dbxl_cd_fold_2_Effective,dbxl_cd_fold_3_Ineffective,dbxl_cd_fold_3_Adequate,dbxl_cd_fold_3_Effective,dbxl_cd_fold_4_Ineffective,dbxl_cd_fold_4_Adequate,dbxl_cd_fold_4_Effective,discourse_idx.5,hf43_dr_fold_0_Ineffective,hf43_dr_fold_0_Adequate,hf43_dr_fold_0_Effective,hf43_dr_fold_1_Ineffective,hf43_dr_fold_1_Adequate,hf43_dr_fold_1_Effective,hf43_dr_fold_2_Ineffective,hf43_dr_fold_2_Adequate,hf43_dr_fold_2_Effective,hf43_dr_fold_3_Ineffective,hf43_dr_fold_3_Adequate,hf43_dr_fold_3_Effective,hf43_dr_fold_4_Ineffective,hf43_dr_fold_4_Adequate,hf43_dr_fold_4_Effective,fold_k_5_seed_42,fold_k_5_seed_2020,fold_k_8_seed_42,fold_k_8_seed_2020,fold_k_10_seed_42,fold_k_10_seed_2020,target,Ineffective,Adequate,Effective,discourse_text,discourse_type,discourse_effectiveness
0,423A1CA112E2,1622627660524,1078c641ef02,0.034966,0.954007,0.011027,0.018977,0.963046,0.017976,0.015324,0.967579,0.017097,0.042056,0.941951,0.015993,0.038436,0.953973,0.007592,1078c641ef02,0.035761,0.953399,0.01084,0.067797,0.895976,0.036227,0.085879,0.883173,0.030948,0.078403,0.907612,0.013985,0.062033,0.927665,0.010302,1078c641ef02,0.032995,0.954974,0.012031,0.032231,0.952640,0.015129,0.042135,0.946509,0.011356,0.015863,0.955661,0.028476,0.019859,0.975816,0.004325,1078c641ef02,0.017759,0.961099,0.021142,0.078091,0.907290,0.014619,0.037290,0.911325,0.051385,0.038027,0.955099,0.006874,0.037068,0.954009,0.008923,1078c641ef02,0.024958,0.960981,0.014062,0.055443,0.936815,0.007742,0.048652,0.947200,0.004149,0.026633,0.949759,0.023609,0.023523,0.967084,0.009393,NaN,0.026505,0.945362,0.028132,0.035739,0.945428,0.018833,0.031681,0.939529,0.028790,0.019637,0.962005,0.018357,0.024594,0.963408,0.011998,4.0,2.0,5.0,4.0,8.0,8.0,1,0.0,1.0,0.0,Modern humans today are always on their phone....,Lead,Ineffective
1,423A1CA112E2,1622627653021,3695c85c6495,0.197380,0.798637,0.003982,0.076309,0.918173,0.005518,0.124825,0.872499,0.002675,0.220555,0.776049,0.003396,0.202561,0.795569,0.001870,3695c85c6495,0.108020,0.889190,0.00279,0.114678,0.881405,0.003916,0.138686,0.855806,0.005508,0.145438,0.850422,0.004140,0.175941,0.820701,0.003357,3695c85c6495,0.056605,0.939123,0.004272,0.077579,0

In [15]:
train_df.loc[train_df[kfold_name]!=10,TARGET].sum()

Ineffective     6461.0
Adequate       20972.0
Effective       9326.0
dtype: float64

In [16]:
df_fold0[kfold_name].value_counts()

10    7381
Name: fold_k_5_seed_42, dtype: int64

In [17]:
train_df[kfold_name].value_counts()

10.0    107528
4.0       7441
0.0       7381
1.0       7343
2.0       7318
3.0       7276
Name: fold_k_5_seed_42, dtype: int64

In [18]:
train_df = pd.concat([train_df,df_fold0],axis=0).reset_index(drop=True)
train_df.shape

(151668, 111)

In [11]:
train_df.shape

(144287, 111)

In [19]:
class args:
    seed = 2022
    
    # Model
    model_name = "microsoft/deberta-large" #"funnel-transformer/large" #"allenai/longformer-large-4096"
    
    # CV
    kfold_name = "fold_k_5_seed_42"
    selected_folds = [0]
    
    # Paths
    data_folder = ROOT_DATA/"feedback-prize-2021"
    name = "deberta_large"
    checkpoints_path = Path(fr'../checkpoint/{kfold_name}/{name}/psl')
    pretrained_path = Path(fr'../checkpoint/MLM/{name}/')
    add_special_tokens = True
    input_type = "cls_end"
    sample = False
    mask_pct = 0
    use_dropout = False
    use_gradient_checkpointing = False
    use_awp = False
    val_score_min = 1.
    grad_clip = False
    add_soft_labels_fold = True
    # Names
    checkpoints_name = 'log'   
    
    model = {
            "max_len":2048,
            "max_len_eval":2048,
            'loss':"nn.CrossEntropyLoss",
            'pretrained_config':Path(fr'../checkpoint/MLM/{name}/config.pth'),
            "pretrained_weights":None,
            "pretrained_tokenizer":Path(fr'../checkpoint/MLM/{name}/tokenizer'),
            "num_labels":3,
            "model_name":model_name
            }
    
    optimizer = {
            "name":"optim.AdamW",
            'params':{"lr":4e-6,'eps':1e-6,"betas":[0.9, 0.999],
                     "weight_decay": 0.01
                     },            
            }

    scheduler = {
            "name":"poly",
            'params':{
                      "lr_end":3e-7,"power":3,"epochs":4
                     },
            "warmup":0.1,            
            }
    
    train_loader = {
            "batch_size":2,
            'drop_last':True,
            "num_workers":15,
            "pin_memory":False,
            "shuffle":True,
            }
    
    val_loader = {
            "batch_size":1,
            'drop_last':False,
            "num_workers":15,
            "pin_memory":False,
            "shuffle":False
            }
    trainer = {"use_amp":True,'epochs':4}
    callbacks = {'save':True,"es":True,"patience":3,
                 'verbose_eval':1,"epoch_pct_eval":1/10,"epoch_eval_dist":"uniforme",
                 "metric_track":"val_loss","mode":"min",'top_k':3,"softmax_before":0
                }
    
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     device = torch.device("cpu")
    
args.checkpoints_path.mkdir(parents=True,exist_ok=True)

In [20]:
from train_utils_awp_psl import kfold,FeedbackModel

In [21]:
kfold(args,train_df)

50279
----------- fold_k_5_seed_42 ---------

-------------   Fold 1 / 6  -------------



  0%|          | 0/15593 [00:00<?, ?it/s]

  0%|          | 0/15593 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (747 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/838 [00:00<?, ?it/s]

...

Using Pretrained Weights
../checkpoint/MLM/deberta_large/_epoch=7 _step=1 _fold=0 _val_loss=1.3127 _val_log_loss=1.3127 _train_loss=1.7996.pth
reseizing in nn
Using Pretrained Weights
../checkpoint/MLM/deberta_large/_epoch=7 _step=1 _fold=0 _val_loss=1.3127 _val_log_loss=1.3127 _train_loss=1.7996.pth
    -> 405180419 trainable parameters

Using Amp


  0%|          | 0/7796 [00:00<?, ?it/s]

[CLS] [cls_lead] You Ġand Ġyour Ġfriends Ġand Ġfamily Ġcould Ġoften Ġhave Ġarguments Ġon Ġwhether Ġsomething Ġis Ġreal Ġor Ġnot , Ġsuch Ġas Ġin Ġmovies . ĠSome Ġmight Ġthink Ġone Ġthing Ġis Ġspecial Ġeffects , Ġothers Ġwill Ġthink Ġit Ġis Ġreal . ĠIn ĠUn mask ing Ġthe ĠFace Ġon ĠMars , Ġthe Ġauthor Ġexplains Ġcontroversy Ġof Ġthe Ġ" face " Ġfound Ġon ĠMars ' Ġsurface . ĠIt Ġbecame Ġan Ġicon Ġonce Ġit Ġwas Ġreleased Ġto Ġthe Ġpublic . ĠIt Ġwas Ġhighlighted Ġfor Ġdecades Ġand Ġthe Ġsociety Ġargued Ġover Ġit Ġbeing Ġan Ġalien Ġmaking , Ġor Ġjust Ġa Ġland Ġform Ġcalled Ġa Ġmes a . [end_lead] Ġ [cls_position] The Ġface Ġwas Ġjust Ġa Ġnatural Ġland form Ġand Ġit Ġshouldn 't Ġhave Ġbeen Ġthought Ġas Ġan Ġalien Ġform . [end_position] ĊĊ [cls_claim] The Ġmost Ġlogical Ġanswer Ġto Ġthis Ġwas Ġthat Ġit Ġwas Ġa Ġmes a , Ġand Ġthat Ġthe Ġshadows Ġon Ġthe Ġsurface Ġhappened Ġto Ġmake Ġit Ġlook Ġlike Ġa Ġhuman Ġface Ġof Ġan Ġe gypt ian Ġph araoh . [end_claim] Ġ [cls_evidence] This Ġstruck Ġcivilizati

  0%|          | 0/7796 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 422.00 MiB (GPU 0; 47.51 GiB total capacity; 36.37 GiB already allocated; 85.94 MiB free; 39.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF